In [1]:
import os
import eikon as ek
import numpy as np
import pickle
import pandas as pd
import cufflinks as cf
import configparser as cp

In [2]:
cfg = cp.ConfigParser()
cfg.read('eikon.cfg')  # adjust for different file location

['eikon.cfg']

In [3]:
ek.set_app_key(cfg['eikon']['app_id']) #set_app_id function being deprecated

## Get a List of CIKs

### From Compustat
#### For Both Bankrupt and Healthy Companies

In [4]:
# set the path to csv files
# path = 'D:\\studyproject\\bankruptcy\\data\\Compustat\\' # for win
path = '/Users/user/Documents/Bankruptcy/bankruptcy/data/Compustat/' # for mac

# read the csv files
bankrupt_compustat = pd.read_csv(path + 'list_bankrupt.csv', dtype=object, index_col=0)
healthy = pd.read_csv(path + 'list_healthy.csv', dtype=object, index_col=0)

bankrupt_compustat.head()

,Identifier,Company,Data Deletion Date,Deletion Reason,Ticker,CUSIP,CIK
0,001367,Amber Resources Company of C,08/31/2012,2.0,3AMBE,023184203,0000276750
1,002033,Fairchild Corp. (The),11/01/2011,2.0,FCHDQ,303698104,0000009779
2,004049,Constar International Inc,06/01/2011,2.0,CNSTQ,21036U206,0000029806
3,004352,Energy Conversion Devices In,09/04/2012,2.0,ENERQ,292659109,0000032878
4,004768,Fleetwood Enterprises Inc.,08/23/2010,2.0,FLTWQ,339099103,0000314132


In [5]:
healthy.head()

,Identifier,Company,Data Deletion Date,Deletion Reason,Ticker,CUSIP,CIK
0,001004,AAR Corp,.,NaN,AIR,000361105,0000001750
1,001013,ADC Telecommunications Inc.,12/10/2010,1.0,ADCT,000886309,0000061478
2,001021,AFP Imaging Corp,09/15/2014,7.0,IWKS,001058205,0000319126
3,001034,Alpharma Inc.,12/31/2008,1.0,ALO.2,020813101,0000730469
4,001045,American Airlines Group Inc,.,NaN,AAL,02376R102,0000006201


In [6]:
bankrupt_compustat_CIK = bankrupt_compustat.CIK.dropna().to_list()
healthy_CIK = healthy.CIK.dropna().to_list()

print(len(bankrupt_compustat_CIK), 'CIKs available for Compustat bankrupt companies, and')
print(len(healthy_CIK), 'CIKs available for healthy companies.')


100 CIKs available for Compustat bankrupt companies, and
14761 CIKs available for healthy companies.


### From Edgar
#### For Bankrupt Companies

In [7]:
# set the path to csv file
# path = 'D:\\studyproject\\bankruptcy\\data\\Edgar\\' # for win
path = '/Users/user/Documents/Bankruptcy/bankruptcy/data/Edgar/' # for mac

# read the csv files
bankrupt_edgar = pd.read_csv(path + 'edgar_bankrupt_companies.csv', dtype=object)

bankrupt_edgar.head()

,CIK,Company name,Bankruptcy chapter,Form type,Date,URL
0,0000002034,ACETO CORP,11,8-K,09/20/2019,https://www.sec.gov/Archives/edgar/data/2034/0...
1,0000002178,"ADAMS RESOURCES & ENERGY, INC.",11,8-K,04/24/2017,https://www.sec.gov/Archives/edgar/data/2178/0...
2,0000004515,AMERICAN AIRLINES INC,11,8-K,11/29/2011,https://www.sec.gov/Archives/edgar/data/4515/0...
3,0000004707,AMERICA CAPITAL CORP,11,8-K,02/09/2009,https://www.sec.gov/Archives/edgar/data/4707/0...
4,0000006201,AMR CORP,11,8-K,10/23/2013,https://www.sec.gov/Archives/edgar/data/6201/0...


In [8]:
bankrupt_edgar_CIK = bankrupt_edgar.CIK.dropna().to_list()

print(len(bankrupt_edgar_CIK), 'CIKs available for Edgar bankrupt companies.')

1075 CIKs available for Edgar bankrupt companies.


## Merging Bankrupt Lists

In [9]:
# add two lists and remove the duplicates
bankrupt_CIK = list(set(bankrupt_compustat_CIK + bankrupt_edgar_CIK))

len(bankrupt_CIK)

1103

****

## Download the Data

In [10]:
# create folders for saving data
if not os.path.exists('data_bankrupt_CIK'):
    os.makedirs('data_bankrupt_CIK')
    
if not os.path.exists('data_healthy_CIK'):
    os.makedirs('data_healthy_CIK')

In [11]:
# Eikon does not work with large lists, so we
# need to break the healthy_CIK list
# and also bankrupt_CIK

bankrupt_CIK_list = [bankrupt_CIK[i:i + 500] for i in range(0, len(bankrupt_CIK), 500)]
healthy_CIK_list = [healthy_CIK[i:i + 500] for i in range(0, len(healthy_CIK), 500)]

print(len(bankrupt_CIK_list))
len(healthy_CIK_list)


3


30

### Financial Factors


In [12]:
# add all financial factors we need to replicate *the paper*

fields = ['TR.CommonName',
          'TR.CIKNUMBER',
          'TR.TotalCurrentAssets', # Current Assets
          'TR.TotalCurrLiabilities', # Current Liabilities
          'TR.AccountsPayableCFStmt', # Accounts Payable
          'TR.CashAndSTInvestments',
          'TR.TotalAssetsReported',
          'TR.CashAndSTInvestments',
          'TR.MarketCapDS',
          'TR.TotalCurrLiabilities',
          'TR.TotalAssetsReported',
          'TR.TotalCurrLiabilities',
          'TR.PCEBIT',
          'TR.DepreciationAmort',
          'TR.TotalAssetsReported',
          'TR.BGS.BusEarningsBeforeInterestandTaxesEbit',
          'TR.BGS.GeoEarningsBeforeInterestandTaxesEbit',
          'TR.TotalAssetsReported',
          'TR.BGS.BusEarningsBeforeInterestandTaxesEbit',
          'TR.BGS.GeoEarningsBeforeInterestandTaxesEbit',
          'TR.TotalDebtOutstanding',
          'TR.NetDebt',
          'TR.TotalAssetsReported',
          'TR.PCTotInvt5YrCompAnnGrowthRate',
          'TR.Inventories',
          'TR.NetSales',
          'TR.TotalCurrLiabilities',
          'TR.TotalAssetsReported',
          'TR.TotalCurrLiabilities',
          'TR.TotalAssetsReported',
          'TR.TotalCurrLiabilities',
          'TR.TotalLiabilities',
          'TR.TotalCurrLiabilities',
          'TR.NetSales',
          'TR.TotalLiabilities',
          'TR.TotalAssetsReported',
          'TR.TotalLiabilities',
          'TR.TotalLiabilities',
          'TR.TangibleBookValueRptd',
          'TR.TangibleBookValueBnk',
          'TR.TangibleBookValueIns',
          'TR.TangibleBookValueUtil',
          'TR.TangibleBVPS',
          'TR.NetIncomeStartingLine',
          'TR.TotalAssetsReported',
          'TR.NetIncomeStartingLine',
          'TR.NetIncomeStartingLine',
          'TR.NetSales',
          'TR.OperatingIncome',
          'TR.TotalAssetsReported',
          'TR.STOCKPRICE',
          'TR.FiPrice',
          'TR.TotalCurrLiabilities',
          'TR.RetainedEarnings',
          'TR.TotalAssetsReported',
          'TR.MARKETCAPITALISATION',
          'TR.TotalEquity',
          'TR.TotalAssetsReported',
          'TR.Cash',
          'TR.FiPricingVolatility',
          'TR.PCWrkgCapPrdOverPrdAvg(Period=FY0)',
          'TR.PCWrkgCapPrdOverPrdPctChg(Period=FY0)',
          'TR.InstrumentIsActive', # bankruptcy check (not reliable)
          'TR.IsDelistedQuote',
          'TR.NAICSIndustryGroup']  # bankruptcy check (not reliable)


print(len(fields))

# remove duplicates
fields = list(set(fields))

print(len(fields))

65
37


# Download Data

## Bankrupt Companies

### 2000

In [14]:
# first chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2000-01-01',
                                             'EDate':'2001-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data1_2000.csv', index=False)


In [15]:
# second chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2000-01-01',
                                             'EDate':'2001-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data2_2000.csv', index=False)

In [16]:
# third chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2000-01-01',
                                             'EDate':'2001-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data3_2000.csv', index=False)

### 2001

In [17]:
# first chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2001-01-01',
                                             'EDate':'2002-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data1_2001.csv', index=False)


In [18]:
# second chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2001-01-01',
                                             'EDate':'2002-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data2_2001.csv', index=False)

In [19]:
# third chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2001-01-01',
                                             'EDate':'2002-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data3_2001.csv', index=False)

### 2002

In [20]:
# first chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2002-01-01',
                                             'EDate':'2003-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data1_2002.csv', index=False)


In [21]:
# second chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2002-01-01',
                                             'EDate':'2003-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data2_2002.csv', index=False)

In [22]:
# third chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2002-01-01',
                                             'EDate':'2003-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data3_2002.csv', index=False)

### 2003

In [23]:
# first chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2003-01-01',
                                             'EDate':'2004-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data1_2003.csv', index=False)


In [24]:
# second chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2003-01-01',
                                             'EDate':'2004-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data2_2003.csv', index=False)

In [25]:
# third chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2003-01-01',
                                             'EDate':'2004-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data3_2003.csv', index=False)

### 2004

In [26]:
# first chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2004-01-01',
                                             'EDate':'2005-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data1_2004.csv', index=False)


In [27]:
# second chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2004-01-01',
                                             'EDate':'2005-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data2_2004.csv', index=False)

In [28]:
# third chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2004-01-01',
                                             'EDate':'2005-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data3_2004.csv', index=False)

### 2005

In [29]:
# first chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2005-01-01',
                                             'EDate':'2006-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data1_2005.csv', index=False)


In [30]:
# second chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2005-01-01',
                                             'EDate':'2006-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data2_2005.csv', index=False)

In [31]:
# third chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2005-01-01',
                                             'EDate':'2006-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data3_2005.csv', index=False)

### 2006

In [32]:
# first chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2006-01-01',
                                             'EDate':'2007-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data1_2006.csv', index=False)


In [33]:
# second chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2006-01-01',
                                             'EDate':'2007-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data2_2006.csv', index=False)

In [34]:
# third chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2006-01-01',
                                             'EDate':'2007-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data3_2006.csv', index=False)

### 2007

In [35]:
# first chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2007-01-01',
                                             'EDate':'2008-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data1_2007.csv', index=False)


In [36]:
# second chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2007-01-01',
                                             'EDate':'2008-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data2_2007.csv', index=False)

In [37]:
# third chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2007-01-01',
                                             'EDate':'2008-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data3_2007.csv', index=False)

### 2008

In [38]:
# first chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2008-01-01',
                                             'EDate':'2009-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data1_2008.csv', index=False)


In [39]:
# second chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2008-01-01',
                                             'EDate':'2009-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data2_2008.csv', index=False)

In [40]:
# third chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2008-01-01',
                                             'EDate':'2009-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data3_2008.csv', index=False)

### 2009

In [41]:
# first chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2009-01-01',
                                             'EDate':'2010-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data1_2009.csv', index=False)


In [42]:
# second chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2009-01-01',
                                             'EDate':'2010-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data2_2009.csv', index=False)

In [43]:
# third chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2009-01-01',
                                             'EDate':'2010-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data3_2009.csv', index=False)

### 2010

In [44]:
# first chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2010-01-01',
                                             'EDate':'2011-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data1_2010.csv', index=False)


In [45]:
# second chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2010-01-01',
                                             'EDate':'2011-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data2_2010.csv', index=False)

In [46]:
# third chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2010-01-01',
                                             'EDate':'2011-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data3_2010.csv', index=False)

### 2011

In [47]:
# first chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2011-01-01',
                                             'EDate':'2012-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data1_2011.csv', index=False)


In [48]:
# second chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2011-01-01',
                                             'EDate':'2012-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data2_2011.csv', index=False)

In [49]:
# third chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2011-01-01',
                                             'EDate':'2012-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data3_2011.csv', index=False)

### 2012

In [50]:
# first chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2012-01-01',
                                             'EDate':'2013-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data1_2012.csv', index=False)


In [51]:
# second chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2012-01-01',
                                             'EDate':'2013-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data2_2012.csv', index=False)

In [52]:
# third chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2012-01-01',
                                             'EDate':'2013-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data3_2012.csv', index=False)

### 2013

In [53]:
# first chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2013-01-01',
                                             'EDate':'2014-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data1_2013.csv', index=False)


In [54]:
# second chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2013-01-01',
                                             'EDate':'2014-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data2_2013.csv', index=False)

In [55]:
# third chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2013-01-01',
                                             'EDate':'2014-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data3_2013.csv', index=False)

### 2014

In [56]:
# first chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2014-01-01',
                                             'EDate':'2015-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data1_2014.csv', index=False)


In [57]:
# second chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2014-01-01',
                                             'EDate':'2015-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data2_2014.csv', index=False)

In [58]:
# third chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2014-01-01',
                                             'EDate':'2015-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data3_2014.csv', index=False)

### 2015

In [59]:
# first chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2015-01-01',
                                             'EDate':'2016-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data1_2015.csv', index=False)


In [60]:
# second chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2015-01-01',
                                             'EDate':'2016-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data2_2015.csv', index=False)

In [61]:
# third chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2015-01-01',
                                             'EDate':'2016-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data3_2015.csv', index=False)

### 2016

In [62]:
# first chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2016-01-01',
                                             'EDate':'2017-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data1_2016.csv', index=False)


In [63]:
# second chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2016-01-01',
                                             'EDate':'2017-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data2_2016.csv', index=False)

In [64]:
# third chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2016-01-01',
                                             'EDate':'2017-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data3_2016.csv', index=False)

### 2017

In [65]:
# first chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2017-01-01',
                                             'EDate':'2018-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data1_2017.csv', index=False)


In [66]:
# second chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2017-01-01',
                                             'EDate':'2018-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data2_2017.csv', index=False)

In [67]:
# third chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2017-01-01',
                                             'EDate':'2018-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data3_2017.csv', index=False)

### 2018

In [68]:
# first chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2018-01-01',
                                             'EDate':'2019-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data1_2018.csv', index=False)


In [69]:
# second chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2018-01-01',
                                             'EDate':'2019-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data2_2018.csv', index=False)

In [70]:
# third chuck of bankrupt data
bankrupt_data, err = ek.get_data(bankrupt_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2018-01-01',
                                             'EDate':'2019-01-01'})
# and save the dataframe as a csv file
bankrupt_data.to_csv('data_bankrupt_CIK/bankrupt_data3_2018.csv', index=False)

****

## Healthy Companies

### 2000

In [71]:
# first chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2000-01-01',
                                             'EDate':'2001-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data01_2000.csv', index=False)


In [72]:
# second chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2000-01-01',
                                             'EDate':'2001-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data02_2000.csv', index=False)


In [73]:
# third chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2000-01-01',
                                             'EDate':'2001-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data03_2000.csv', index=False)


In [74]:
# fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[3], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2000-01-01',
                                             'EDate':'2001-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data04_2000.csv', index=False)


In [75]:
# fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[4], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2000-01-01',
                                             'EDate':'2001-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data05_2000.csv', index=False)


In [76]:
# sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[5], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2000-01-01',
                                             'EDate':'2001-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data06_2000.csv', index=False)


In [77]:
# seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[6], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2000-01-01',
                                             'EDate':'2001-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data07_2000.csv', index=False)


In [78]:
# eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[7], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2000-01-01',
                                             'EDate':'2001-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data08_2000.csv', index=False)


In [79]:
# ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[8], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2000-01-01',
                                             'EDate':'2001-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data09_2000.csv', index=False)


In [80]:
# tenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[9], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2000-01-01',
                                             'EDate':'2001-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data10_2000.csv', index=False)


In [81]:
# eleventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[10], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2000-01-01',
                                             'EDate':'2001-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data11_2000.csv', index=False)


In [82]:
# twelfth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[11], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2000-01-01',
                                             'EDate':'2001-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data12_2000.csv', index=False)


In [83]:
# thirteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[12], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2000-01-01',
                                             'EDate':'2001-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data13_2000.csv', index=False)


In [84]:
# fourteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[13], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2000-01-01',
                                             'EDate':'2001-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data14_2000.csv', index=False)


In [85]:
# fifteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[14], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2000-01-01',
                                             'EDate':'2001-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data15_2000.csv', index=False)


In [86]:
# sixteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[15], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2000-01-01',
                                             'EDate':'2001-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data16_2000.csv', index=False)


In [87]:
# seventeenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[16], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2000-01-01',
                                             'EDate':'2001-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data17_2000.csv', index=False)


In [88]:
# eighteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[17], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2000-01-01',
                                             'EDate':'2001-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data18_2000.csv', index=False)


In [89]:
# nineteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[18], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2000-01-01',
                                             'EDate':'2001-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data19_2000.csv', index=False)


In [90]:
# twentieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[19], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2000-01-01',
                                             'EDate':'2001-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data20_2000.csv', index=False)


In [91]:
# twenty-first chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[20], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2000-01-01',
                                             'EDate':'2001-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data21_2000.csv', index=False)


In [92]:
# twenty-second chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[21], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2000-01-01',
                                             'EDate':'2001-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data22_2000.csv', index=False)


In [93]:
# twenty-third chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[22], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2000-01-01',
                                             'EDate':'2001-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data23_2000.csv', index=False)


In [94]:
# twenty-fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[23], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2000-01-01',
                                             'EDate':'2001-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data24_2000.csv', index=False)


In [95]:
# twenty-fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[24], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2000-01-01',
                                             'EDate':'2001-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data25_2000.csv', index=False)


In [96]:
# twenty-sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[25], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2000-01-01',
                                             'EDate':'2001-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data26_2000.csv', index=False)


In [97]:
# twenty-seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[26], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2000-01-01',
                                             'EDate':'2001-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data27_2000.csv', index=False)


In [98]:
# twenty-eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[27], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2000-01-01',
                                             'EDate':'2001-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data28_2000.csv', index=False)


In [99]:
# twenty-ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[28], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2000-01-01',
                                             'EDate':'2001-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data29_2000.csv', index=False)


In [100]:
# thirtieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[29], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2000-01-01',
                                             'EDate':'2001-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data30_2000.csv', index=False)


### 2001

In [101]:
# first chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2001-01-01',
                                             'EDate':'2002-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data01_2001.csv', index=False)


In [102]:
# second chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2001-01-01',
                                             'EDate':'2002-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data02_2001.csv', index=False)


In [103]:
# third chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2001-01-01',
                                             'EDate':'2002-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data03_2001.csv', index=False)


In [104]:
# fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[3], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2001-01-01',
                                             'EDate':'2002-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data04_2001.csv', index=False)


In [105]:
# fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[4], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2001-01-01',
                                             'EDate':'2002-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data05_2001.csv', index=False)


In [106]:
# sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[5], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2001-01-01',
                                             'EDate':'2002-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data06_2001.csv', index=False)


In [107]:
# seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[6], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2001-01-01',
                                             'EDate':'2002-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data07_2001.csv', index=False)


In [108]:
# eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[7], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2001-01-01',
                                             'EDate':'2002-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data08_2001.csv', index=False)


In [109]:
# ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[8], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2001-01-01',
                                             'EDate':'2002-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data09_2001.csv', index=False)


In [110]:
# tenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[9], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2001-01-01',
                                             'EDate':'2002-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data10_2001.csv', index=False)


In [111]:
# eleventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[10], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2001-01-01',
                                             'EDate':'2002-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data11_2001.csv', index=False)


In [112]:
# twelfth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[11], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2001-01-01',
                                             'EDate':'2002-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data12_2001.csv', index=False)


In [113]:
# thirteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[12], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2001-01-01',
                                             'EDate':'2002-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data13_2001.csv', index=False)


In [114]:
# fourteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[13], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2001-01-01',
                                             'EDate':'2002-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data14_2001.csv', index=False)


In [115]:
# fifteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[14], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2001-01-01',
                                             'EDate':'2002-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data15_2001.csv', index=False)


In [116]:
# sixteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[15], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2001-01-01',
                                             'EDate':'2002-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data16_2001.csv', index=False)


In [117]:
# seventeenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[16], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2001-01-01',
                                             'EDate':'2002-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data17_2001.csv', index=False)


In [118]:
# eighteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[17], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2001-01-01',
                                             'EDate':'2002-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data18_2001.csv', index=False)


In [119]:
# nineteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[18], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2001-01-01',
                                             'EDate':'2002-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data19_2001.csv', index=False)


In [120]:
# twentieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[19], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2001-01-01',
                                             'EDate':'2002-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data20_2001.csv', index=False)


In [121]:
# twenty-first chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[20], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2001-01-01',
                                             'EDate':'2002-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data21_2001.csv', index=False)


In [122]:
# twenty-second chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[21], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2001-01-01',
                                             'EDate':'2002-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data22_2001.csv', index=False)


In [123]:
# twenty-third chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[22], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2001-01-01',
                                             'EDate':'2002-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data23_2001.csv', index=False)


In [124]:
# twenty-fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[23], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2001-01-01',
                                             'EDate':'2002-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data24_2001.csv', index=False)


In [125]:
# twenty-fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[24], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2001-01-01',
                                             'EDate':'2002-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data25_2001.csv', index=False)


In [126]:
# twenty-sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[25], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2001-01-01',
                                             'EDate':'2002-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data26_2001.csv', index=False)


In [127]:
# twenty-seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[26], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2001-01-01',
                                             'EDate':'2002-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data27_2001.csv', index=False)


In [128]:
# twenty-eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[27], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2001-01-01',
                                             'EDate':'2002-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data28_2001.csv', index=False)


In [129]:
# twenty-ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[28], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2001-01-01',
                                             'EDate':'2002-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data29_2001.csv', index=False)


In [130]:
# thirtieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[29], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2001-01-01',
                                             'EDate':'2002-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data30_2001.csv', index=False)


### 2002

In [131]:
# first chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2002-01-01',
                                             'EDate':'2003-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data01_2002.csv', index=False)


In [132]:
# second chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2002-01-01',
                                             'EDate':'2003-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data02_2002.csv', index=False)


In [133]:
# third chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2002-01-01',
                                             'EDate':'2003-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data03_2002.csv', index=False)


In [134]:
# fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[3], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2002-01-01',
                                             'EDate':'2003-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data04_2002.csv', index=False)


In [135]:
# fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[4], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2002-01-01',
                                             'EDate':'2003-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data05_2002.csv', index=False)


In [136]:
# sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[5], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2002-01-01',
                                             'EDate':'2003-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data06_2002.csv', index=False)


In [137]:
# seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[6], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2002-01-01',
                                             'EDate':'2003-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data07_2002.csv', index=False)


In [138]:
# eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[7], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2002-01-01',
                                             'EDate':'2003-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data08_2002.csv', index=False)


In [139]:
# ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[8], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2002-01-01',
                                             'EDate':'2003-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data09_2002.csv', index=False)


In [140]:
# tenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[9], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2002-01-01',
                                             'EDate':'2003-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data10_2002.csv', index=False)


In [141]:
# eleventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[10], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2002-01-01',
                                             'EDate':'2003-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data11_2002.csv', index=False)


In [142]:
# twelfth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[11], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2002-01-01',
                                             'EDate':'2003-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data12_2002.csv', index=False)


In [143]:
# thirteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[12], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2002-01-01',
                                             'EDate':'2003-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data13_2002.csv', index=False)


In [144]:
# fourteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[13], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2002-01-01',
                                             'EDate':'2003-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data14_2002.csv', index=False)


In [145]:
# fifteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[14], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2002-01-01',
                                             'EDate':'2003-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data15_2002.csv', index=False)


In [146]:
# sixteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[15], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2002-01-01',
                                             'EDate':'2003-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data16_2002.csv', index=False)


In [147]:
# seventeenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[16], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2002-01-01',
                                             'EDate':'2003-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data17_2002.csv', index=False)


In [148]:
# eighteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[17], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2002-01-01',
                                             'EDate':'2003-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data18_2002.csv', index=False)


In [149]:
# nineteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[18], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2002-01-01',
                                             'EDate':'2003-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data19_2002.csv', index=False)


In [150]:
# twentieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[19], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2002-01-01',
                                             'EDate':'2003-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data20_2002.csv', index=False)


In [151]:
# twenty-first chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[20], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2002-01-01',
                                             'EDate':'2003-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data21_2002.csv', index=False)


In [152]:
# twenty-second chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[21], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2002-01-01',
                                             'EDate':'2003-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data22_2002.csv', index=False)


In [153]:
# twenty-third chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[22], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2002-01-01',
                                             'EDate':'2003-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data23_2002.csv', index=False)


In [154]:
# twenty-fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[23], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2002-01-01',
                                             'EDate':'2003-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data24_2002.csv', index=False)


In [155]:
# twenty-fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[24], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2002-01-01',
                                             'EDate':'2003-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data25_2002.csv', index=False)


In [156]:
# twenty-sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[25], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2002-01-01',
                                             'EDate':'2003-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data26_2002.csv', index=False)


In [157]:
# twenty-seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[26], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2002-01-01',
                                             'EDate':'2003-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data27_2002.csv', index=False)


In [158]:
# twenty-eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[27], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2002-01-01',
                                             'EDate':'2003-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data28_2002.csv', index=False)


In [159]:
# twenty-ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[28], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2002-01-01',
                                             'EDate':'2003-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data29_2002.csv', index=False)


In [160]:
# thirtieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[29], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2002-01-01',
                                             'EDate':'2003-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data30_2002.csv', index=False)


### 2003

In [161]:
# first chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2003-01-01',
                                             'EDate':'2004-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data01_2003.csv', index=False)


In [162]:
# second chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2003-01-01',
                                             'EDate':'2004-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data02_2003.csv', index=False)


In [163]:
# third chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2003-01-01',
                                             'EDate':'2004-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data03_2003.csv', index=False)


In [164]:
# fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[3], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2003-01-01',
                                             'EDate':'2004-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data04_2003.csv', index=False)


In [165]:
# fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[4], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2003-01-01',
                                             'EDate':'2004-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data05_2003.csv', index=False)


In [166]:
# sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[5], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2003-01-01',
                                             'EDate':'2004-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data06_2003.csv', index=False)


In [167]:
# seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[6], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2003-01-01',
                                             'EDate':'2004-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data07_2003.csv', index=False)


In [168]:
# eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[7], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2003-01-01',
                                             'EDate':'2004-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data08_2003.csv', index=False)


In [169]:
# ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[8], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2003-01-01',
                                             'EDate':'2004-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data09_2003.csv', index=False)


In [170]:
# tenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[9], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2003-01-01',
                                             'EDate':'2004-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data10_2003.csv', index=False)


In [171]:
# eleventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[10], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2003-01-01',
                                             'EDate':'2004-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data11_2003.csv', index=False)


In [172]:
# twelfth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[11], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2003-01-01',
                                             'EDate':'2004-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data12_2003.csv', index=False)


In [173]:
# thirteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[12], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2003-01-01',
                                             'EDate':'2004-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data13_2003.csv', index=False)


In [174]:
# fourteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[13], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2003-01-01',
                                             'EDate':'2004-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data14_2003.csv', index=False)


In [175]:
# fifteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[14], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2003-01-01',
                                             'EDate':'2004-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data15_2003.csv', index=False)


In [176]:
# sixteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[15], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2003-01-01',
                                             'EDate':'2004-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data16_2003.csv', index=False)


In [177]:
# seventeenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[16], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2003-01-01',
                                             'EDate':'2004-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data17_2003.csv', index=False)


In [178]:
# eighteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[17], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2003-01-01',
                                             'EDate':'2004-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data18_2003.csv', index=False)


In [179]:
# nineteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[18], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2003-01-01',
                                             'EDate':'2004-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data19_2003.csv', index=False)


In [180]:
# twentieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[19], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2003-01-01',
                                             'EDate':'2004-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data20_2003.csv', index=False)


In [181]:
# twenty-first chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[20], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2003-01-01',
                                             'EDate':'2004-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data21_2003.csv', index=False)


In [182]:
# twenty-second chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[21], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2003-01-01',
                                             'EDate':'2004-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data22_2003.csv', index=False)


In [183]:
# twenty-third chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[22], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2003-01-01',
                                             'EDate':'2004-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data23_2003.csv', index=False)


In [184]:
# twenty-fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[23], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2003-01-01',
                                             'EDate':'2004-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data24_2003.csv', index=False)


In [185]:
# twenty-fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[24], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2003-01-01',
                                             'EDate':'2004-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data25_2003.csv', index=False)


In [186]:
# twenty-sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[25], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2003-01-01',
                                             'EDate':'2004-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data26_2003.csv', index=False)


In [187]:
# twenty-seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[26], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2003-01-01',
                                             'EDate':'2004-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data27_2003.csv', index=False)


In [188]:
# twenty-eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[27], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2003-01-01',
                                             'EDate':'2004-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data28_2003.csv', index=False)


In [189]:
# twenty-ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[28], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2003-01-01',
                                             'EDate':'2004-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data29_2003.csv', index=False)


# RUN THE FOLLOWING CELL

In [ ]:
# thirtieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[29], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2003-01-01',
                                             'EDate':'2004-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data30_2003.csv', index=False)


# RUN THE CELL ABOVE

### 2004

In [ ]:
# first chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2004-01-01',
                                             'EDate':'2005-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data01_2004.csv', index=False)


In [ ]:
# second chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2004-01-01',
                                             'EDate':'2005-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data02_2004.csv', index=False)


In [ ]:
# third chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2004-01-01',
                                             'EDate':'2005-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data03_2004.csv', index=False)


In [ ]:
# fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[3], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2004-01-01',
                                             'EDate':'2005-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data04_2004.csv', index=False)


In [ ]:
# fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[4], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2004-01-01',
                                             'EDate':'2005-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data05_2004.csv', index=False)


In [ ]:
# sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[5], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2004-01-01',
                                             'EDate':'2005-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data06_2004.csv', index=False)


In [ ]:
# seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[6], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2004-01-01',
                                             'EDate':'2005-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data07_2004.csv', index=False)


In [ ]:
# eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[7], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2004-01-01',
                                             'EDate':'2005-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data08_2004.csv', index=False)


In [ ]:
# ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[8], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2004-01-01',
                                             'EDate':'2005-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data09_2004.csv', index=False)


In [ ]:
# tenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[9], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2004-01-01',
                                             'EDate':'2005-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data10_2004.csv', index=False)


In [ ]:
# eleventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[10], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2004-01-01',
                                             'EDate':'2005-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data11_2004.csv', index=False)


In [ ]:
# twelfth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[11], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2004-01-01',
                                             'EDate':'2005-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data12_2004.csv', index=False)


In [ ]:
# thirteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[12], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2004-01-01',
                                             'EDate':'2005-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data13_2004.csv', index=False)


In [ ]:
# fourteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[13], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2004-01-01',
                                             'EDate':'2005-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data14_2004.csv', index=False)


In [ ]:
# fifteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[14], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2004-01-01',
                                             'EDate':'2005-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data15_2004.csv', index=False)


In [ ]:
# sixteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[15], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2004-01-01',
                                             'EDate':'2005-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data16_2004.csv', index=False)


In [ ]:
# seventeenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[16], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2004-01-01',
                                             'EDate':'2005-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data17_2004.csv', index=False)


In [ ]:
# eighteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[17], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2004-01-01',
                                             'EDate':'2005-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data18_2004.csv', index=False)


In [ ]:
# nineteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[18], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2004-01-01',
                                             'EDate':'2005-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data19_2004.csv', index=False)


In [ ]:
# twentieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[19], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2004-01-01',
                                             'EDate':'2005-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data20_2004.csv', index=False)


In [ ]:
# twenty-first chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[20], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2004-01-01',
                                             'EDate':'2005-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data21_2004.csv', index=False)


In [ ]:
# twenty-second chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[21], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2004-01-01',
                                             'EDate':'2005-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data22_2004.csv', index=False)


In [ ]:
# twenty-third chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[22], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2004-01-01',
                                             'EDate':'2005-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data23_2004.csv', index=False)


In [ ]:
# twenty-fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[23], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2004-01-01',
                                             'EDate':'2005-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data24_2004.csv', index=False)


In [ ]:
# twenty-fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[24], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2004-01-01',
                                             'EDate':'2005-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data25_2004.csv', index=False)


In [ ]:
# twenty-sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[25], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2004-01-01',
                                             'EDate':'2005-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data26_2004.csv', index=False)


In [ ]:
# twenty-seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[26], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2004-01-01',
                                             'EDate':'2005-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data27_2004.csv', index=False)


In [ ]:
# twenty-eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[27], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2004-01-01',
                                             'EDate':'2005-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data28_2004.csv', index=False)


In [ ]:
# twenty-ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[28], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2004-01-01',
                                             'EDate':'2005-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data29_2004.csv', index=False)


In [ ]:
# thirtieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[29], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2004-01-01',
                                             'EDate':'2005-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data30_2004.csv', index=False)


In [ ]:
# thirtieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[29], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2004-01-01',
                                             'EDate':'2005-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data30_2004.csv', index=False)


### 2005

In [ ]:
# first chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2005-01-01',
                                             'EDate':'2006-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data01_2005.csv', index=False)


In [ ]:
# second chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2005-01-01',
                                             'EDate':'2006-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data02_2005.csv', index=False)


In [ ]:
# third chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2005-01-01',
                                             'EDate':'2006-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data03_2005.csv', index=False)


In [ ]:
# fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[3], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2005-01-01',
                                             'EDate':'2006-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data04_2005.csv', index=False)


In [ ]:
# fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[4], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2005-01-01',
                                             'EDate':'2006-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data05_2005.csv', index=False)


In [ ]:
# sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[5], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2005-01-01',
                                             'EDate':'2006-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data06_2005.csv', index=False)


In [ ]:
# seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[6], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2005-01-01',
                                             'EDate':'2006-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data07_2005.csv', index=False)


In [ ]:
# eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[7], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2005-01-01',
                                             'EDate':'2006-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data08_2005.csv', index=False)


In [ ]:
# ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[8], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2005-01-01',
                                             'EDate':'2006-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data09_2005.csv', index=False)


In [ ]:
# tenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[9], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2005-01-01',
                                             'EDate':'2006-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data10_2005.csv', index=False)


In [ ]:
# eleventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[10], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2005-01-01',
                                             'EDate':'2006-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data11_2005.csv', index=False)


In [ ]:
# twelfth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[11], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2005-01-01',
                                             'EDate':'2006-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data12_2005.csv', index=False)


In [ ]:
# thirteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[12], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2005-01-01',
                                             'EDate':'2006-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data13_2005.csv', index=False)


In [ ]:
# fourteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[13], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2005-01-01',
                                             'EDate':'2006-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data14_2005.csv', index=False)


In [ ]:
# fifteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[14], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2005-01-01',
                                             'EDate':'2006-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data15_2005.csv', index=False)


In [ ]:
# sixteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[15], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2005-01-01',
                                             'EDate':'2006-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data16_2005.csv', index=False)


In [ ]:
# seventeenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[16], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2005-01-01',
                                             'EDate':'2006-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data17_2005.csv', index=False)


In [ ]:
# eighteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[17], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2005-01-01',
                                             'EDate':'2006-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data18_2005.csv', index=False)


In [ ]:
# nineteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[18], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2005-01-01',
                                             'EDate':'2006-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data19_2005.csv', index=False)


In [ ]:
# twentieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[19], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2005-01-01',
                                             'EDate':'2006-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data20_2005.csv', index=False)


In [ ]:
# twenty-first chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[20], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2005-01-01',
                                             'EDate':'2006-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data21_2005.csv', index=False)


In [ ]:
# twenty-second chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[21], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2005-01-01',
                                             'EDate':'2006-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data22_2005.csv', index=False)


In [ ]:
# twenty-third chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[22], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2005-01-01',
                                             'EDate':'2006-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data23_2005.csv', index=False)


In [ ]:
# twenty-fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[23], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2005-01-01',
                                             'EDate':'2006-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data24_2005.csv', index=False)


In [ ]:
# twenty-fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[24], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2005-01-01',
                                             'EDate':'2006-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data25_2005.csv', index=False)


In [ ]:
# twenty-sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[25], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2005-01-01',
                                             'EDate':'2006-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data26_2005.csv', index=False)


In [ ]:
# twenty-seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[26], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2005-01-01',
                                             'EDate':'2006-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data27_2005.csv', index=False)


In [ ]:
# twenty-eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[27], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2005-01-01',
                                             'EDate':'2006-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data28_2005.csv', index=False)


In [ ]:
# twenty-ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[28], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2005-01-01',
                                             'EDate':'2006-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data29_2005.csv', index=False)


In [ ]:
# thirtieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[29], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2005-01-01',
                                             'EDate':'2006-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data30_2005.csv', index=False)


### 2006

In [ ]:
# first chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2006-01-01',
                                             'EDate':'2007-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data01_2006.csv', index=False)


In [ ]:
# second chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2006-01-01',
                                             'EDate':'2007-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data02_2006.csv', index=False)


In [ ]:
# third chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2006-01-01',
                                             'EDate':'2007-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data03_2006.csv', index=False)


In [ ]:
# fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[3], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2006-01-01',
                                             'EDate':'2007-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data04_2006.csv', index=False)


In [ ]:
# fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[4], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2006-01-01',
                                             'EDate':'2007-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data05_2006.csv', index=False)


In [ ]:
# sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[5], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2006-01-01',
                                             'EDate':'2007-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data06_2006.csv', index=False)


In [ ]:
# seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[6], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2006-01-01',
                                             'EDate':'2007-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data07_2006.csv', index=False)


In [ ]:
# eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[7], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2006-01-01',
                                             'EDate':'2007-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data08_2006.csv', index=False)


In [ ]:
# ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[8], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2006-01-01',
                                             'EDate':'2007-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data09_2006.csv', index=False)


In [ ]:
# tenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[9], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2006-01-01',
                                             'EDate':'2007-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data10_2006.csv', index=False)


In [ ]:
# eleventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[10], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2006-01-01',
                                             'EDate':'2007-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data11_2006.csv', index=False)


In [ ]:
# twelfth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[11], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2006-01-01',
                                             'EDate':'2007-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data12_2006.csv', index=False)


In [ ]:
# thirteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[12], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2006-01-01',
                                             'EDate':'2007-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data13_2006.csv', index=False)


In [ ]:
# fourteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[13], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2006-01-01',
                                             'EDate':'2007-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data14_2006.csv', index=False)


In [ ]:
# fifteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[14], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2006-01-01',
                                             'EDate':'2007-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data15_2006.csv', index=False)


In [ ]:
# sixteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[15], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2006-01-01',
                                             'EDate':'2007-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data16_2006.csv', index=False)


In [ ]:
# seventeenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[16], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2006-01-01',
                                             'EDate':'2007-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data17_2006.csv', index=False)


In [ ]:
# eighteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[17], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2006-01-01',
                                             'EDate':'2007-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data18_2006.csv', index=False)


In [ ]:
# nineteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[18], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2006-01-01',
                                             'EDate':'2007-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data19_2006.csv', index=False)


In [ ]:
# twentieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[19], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2006-01-01',
                                             'EDate':'2007-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data20_2006.csv', index=False)


In [ ]:
# twenty-first chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[20], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2006-01-01',
                                             'EDate':'2007-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data21_2006.csv', index=False)


In [ ]:
# twenty-second chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[21], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2006-01-01',
                                             'EDate':'2007-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data22_2006.csv', index=False)


In [ ]:
# twenty-third chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[22], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2006-01-01',
                                             'EDate':'2007-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data23_2006.csv', index=False)


In [ ]:
# twenty-fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[23], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2006-01-01',
                                             'EDate':'2007-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data24_2006.csv', index=False)


In [ ]:
# twenty-fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[24], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2006-01-01',
                                             'EDate':'2007-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data25_2006.csv', index=False)


In [ ]:
# twenty-sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[25], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2006-01-01',
                                             'EDate':'2007-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data26_2006.csv', index=False)


In [ ]:
# twenty-seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[26], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2006-01-01',
                                             'EDate':'2007-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data27_2006.csv', index=False)


In [ ]:
# twenty-eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[27], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2006-01-01',
                                             'EDate':'2007-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data28_2006.csv', index=False)


In [ ]:
# twenty-ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[28], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2006-01-01',
                                             'EDate':'2007-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data29_2006.csv', index=False)


In [ ]:
# thirtieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[29], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2006-01-01',
                                             'EDate':'2007-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data30_2006.csv', index=False)


### 2007

In [ ]:
# first chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2007-01-01',
                                             'EDate':'2008-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data01_2007.csv', index=False)


In [ ]:
# second chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2007-01-01',
                                             'EDate':'2008-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data02_2007.csv', index=False)


In [ ]:
# third chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2007-01-01',
                                             'EDate':'2008-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data03_2007.csv', index=False)


In [ ]:
# fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[3], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2007-01-01',
                                             'EDate':'2008-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data04_2007.csv', index=False)


In [ ]:
# fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[4], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2007-01-01',
                                             'EDate':'2008-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data05_2007.csv', index=False)


In [ ]:
# sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[5], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2007-01-01',
                                             'EDate':'2008-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data06_2007.csv', index=False)


In [ ]:
# seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[6], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2007-01-01',
                                             'EDate':'2008-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data07_2007.csv', index=False)


In [ ]:
# eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[7], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2007-01-01',
                                             'EDate':'2008-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data08_2007.csv', index=False)


In [ ]:
# ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[8], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2007-01-01',
                                             'EDate':'2008-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data09_2007.csv', index=False)


In [ ]:
# tenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[9], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2007-01-01',
                                             'EDate':'2008-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data10_2007.csv', index=False)


In [ ]:
# eleventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[10], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2007-01-01',
                                             'EDate':'2008-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data11_2007.csv', index=False)


In [ ]:
# twelfth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[11], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2007-01-01',
                                             'EDate':'2008-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data12_2007.csv', index=False)


In [ ]:
# thirteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[12], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2007-01-01',
                                             'EDate':'2008-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data13_2007.csv', index=False)


In [ ]:
# fourteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[13], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2007-01-01',
                                             'EDate':'2008-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data14_2007.csv', index=False)


In [ ]:
# fifteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[14], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2007-01-01',
                                             'EDate':'2008-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data15_2007.csv', index=False)


In [ ]:
# sixteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[15], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2007-01-01',
                                             'EDate':'2008-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data16_2007.csv', index=False)


In [ ]:
# seventeenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[16], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2007-01-01',
                                             'EDate':'2008-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data17_2007.csv', index=False)


In [ ]:
# eighteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[17], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2007-01-01',
                                             'EDate':'2008-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data18_2007.csv', index=False)


In [ ]:
# nineteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[18], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2007-01-01',
                                             'EDate':'2008-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data19_2007.csv', index=False)


In [ ]:
# twentieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[19], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2007-01-01',
                                             'EDate':'2008-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data20_2007.csv', index=False)


In [ ]:
# twenty-first chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[20], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2007-01-01',
                                             'EDate':'2008-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data21_2007.csv', index=False)


In [ ]:
# twenty-second chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[21], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2007-01-01',
                                             'EDate':'2008-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data22_2007.csv', index=False)


In [ ]:
# twenty-third chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[22], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2007-01-01',
                                             'EDate':'2008-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data23_2007.csv', index=False)


In [ ]:
# twenty-fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[23], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2007-01-01',
                                             'EDate':'2008-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data24_2007.csv', index=False)


In [ ]:
# twenty-fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[24], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2007-01-01',
                                             'EDate':'2008-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data25_2007.csv', index=False)


In [ ]:
# twenty-sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[25], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2007-01-01',
                                             'EDate':'2008-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data26_2007.csv', index=False)


In [ ]:
# twenty-seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[26], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2007-01-01',
                                             'EDate':'2008-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data27_2007.csv', index=False)


In [ ]:
# twenty-eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[27], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2007-01-01',
                                             'EDate':'2008-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data28_2007.csv', index=False)


In [ ]:
# twenty-ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[28], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2007-01-01',
                                             'EDate':'2008-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data29_2007.csv', index=False)


In [ ]:
# thirtieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[29], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2007-01-01',
                                             'EDate':'2008-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data30_2007.csv', index=False)


### 2008

In [ ]:
# first chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2008-01-01',
                                             'EDate':'2009-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data01_2008.csv', index=False)


In [ ]:
# second chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2008-01-01',
                                             'EDate':'2009-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data02_2008.csv', index=False)


In [ ]:
# third chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2008-01-01',
                                             'EDate':'2009-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data03_2008.csv', index=False)


In [ ]:
# fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[3], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2008-01-01',
                                             'EDate':'2009-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data04_2008.csv', index=False)


In [ ]:
# fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[4], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2008-01-01',
                                             'EDate':'2009-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data05_2008.csv', index=False)


In [ ]:
# sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[5], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2008-01-01',
                                             'EDate':'2009-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data06_2008.csv', index=False)


In [ ]:
# seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[6], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2008-01-01',
                                             'EDate':'2009-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data07_2008.csv', index=False)


In [ ]:
# eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[7], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2008-01-01',
                                             'EDate':'2009-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data08_2008.csv', index=False)


In [ ]:
# ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[8], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2008-01-01',
                                             'EDate':'2009-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data09_2008.csv', index=False)


In [ ]:
# tenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[9], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2008-01-01',
                                             'EDate':'2009-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data10_2008.csv', index=False)


In [ ]:
# eleventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[10], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2008-01-01',
                                             'EDate':'2009-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data11_2008.csv', index=False)


In [ ]:
# twelfth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[11], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2008-01-01',
                                             'EDate':'2009-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data12_2008.csv', index=False)


In [ ]:
# thirteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[12], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2008-01-01',
                                             'EDate':'2009-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data13_2008.csv', index=False)


In [ ]:
# fourteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[13], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2008-01-01',
                                             'EDate':'2009-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data14_2008.csv', index=False)


In [ ]:
# fifteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[14], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2008-01-01',
                                             'EDate':'2009-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data15_2008.csv', index=False)


In [ ]:
# sixteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[15], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2008-01-01',
                                             'EDate':'2009-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data16_2008.csv', index=False)


In [ ]:
# seventeenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[16], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2008-01-01',
                                             'EDate':'2009-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data17_2008.csv', index=False)


In [ ]:
# eighteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[17], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2008-01-01',
                                             'EDate':'2009-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data18_2008.csv', index=False)


In [ ]:
# nineteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[18], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2008-01-01',
                                             'EDate':'2009-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data19_2008.csv', index=False)


In [ ]:
# twentieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[19], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2008-01-01',
                                             'EDate':'2009-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data20_2008.csv', index=False)


In [ ]:
# twenty-first chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[20], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2008-01-01',
                                             'EDate':'2009-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data21_2008.csv', index=False)


In [ ]:
# twenty-second chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[21], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2008-01-01',
                                             'EDate':'2009-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data22_2008.csv', index=False)


In [ ]:
# twenty-third chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[22], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2008-01-01',
                                             'EDate':'2009-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data23_2008.csv', index=False)


In [ ]:
# twenty-fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[23], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2008-01-01',
                                             'EDate':'2009-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data24_2008.csv', index=False)


In [ ]:
# twenty-fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[24], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2008-01-01',
                                             'EDate':'2009-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data25_2008.csv', index=False)


In [ ]:
# twenty-sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[25], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2008-01-01',
                                             'EDate':'2009-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data26_2008.csv', index=False)


In [ ]:
# twenty-seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[26], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2008-01-01',
                                             'EDate':'2009-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data27_2008.csv', index=False)


In [ ]:
# twenty-eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[27], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2008-01-01',
                                             'EDate':'2009-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data28_2008.csv', index=False)


In [ ]:
# twenty-ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[28], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2008-01-01',
                                             'EDate':'2009-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data29_2008.csv', index=False)


In [ ]:
# thirtieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[29], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2008-01-01',
                                             'EDate':'2009-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data30_2008.csv', index=False)


### 2009

In [ ]:
# first chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2009-01-01',
                                             'EDate':'2010-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data01_2009.csv', index=False)


In [ ]:
# second chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2009-01-01',
                                             'EDate':'2010-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data02_2009.csv', index=False)


In [ ]:
# third chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2009-01-01',
                                             'EDate':'2010-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data03_2009.csv', index=False)


In [ ]:
# fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[3], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2009-01-01',
                                             'EDate':'2010-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data04_2009.csv', index=False)


In [ ]:
# fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[4], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2009-01-01',
                                             'EDate':'2010-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data05_2009.csv', index=False)


In [ ]:
# sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[5], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2009-01-01',
                                             'EDate':'2010-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data06_2009.csv', index=False)


In [ ]:
# seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[6], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2009-01-01',
                                             'EDate':'2010-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data07_2009.csv', index=False)


In [ ]:
# eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[7], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2009-01-01',
                                             'EDate':'2010-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data08_2009.csv', index=False)


In [ ]:
# ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[8], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2009-01-01',
                                             'EDate':'2010-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data09_2009.csv', index=False)


In [ ]:
# tenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[9], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2009-01-01',
                                             'EDate':'2010-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data10_2009.csv', index=False)


In [ ]:
# eleventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[10], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2009-01-01',
                                             'EDate':'2010-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data11_2009.csv', index=False)


In [ ]:
# twelfth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[11], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2009-01-01',
                                             'EDate':'2010-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data12_2009.csv', index=False)


In [ ]:
# thirteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[12], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2009-01-01',
                                             'EDate':'2010-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data13_2009.csv', index=False)


In [ ]:
# fourteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[13], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2009-01-01',
                                             'EDate':'2010-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data14_2009.csv', index=False)


In [ ]:
# fifteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[14], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2009-01-01',
                                             'EDate':'2010-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data15_2009.csv', index=False)


In [ ]:
# sixteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[15], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2009-01-01',
                                             'EDate':'2010-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data16_2009.csv', index=False)


In [ ]:
# seventeenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[16], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2009-01-01',
                                             'EDate':'2010-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data17_2009.csv', index=False)


In [ ]:
# eighteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[17], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2009-01-01',
                                             'EDate':'2010-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data18_2009.csv', index=False)


In [ ]:
# nineteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[18], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2009-01-01',
                                             'EDate':'2010-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data19_2009.csv', index=False)


In [ ]:
# twentieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[19], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2009-01-01',
                                             'EDate':'2010-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data20_2009.csv', index=False)


In [ ]:
# twenty-first chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[20], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2009-01-01',
                                             'EDate':'2010-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data21_2009.csv', index=False)


In [ ]:
# twenty-second chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[21], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2009-01-01',
                                             'EDate':'2010-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data22_2009.csv', index=False)


In [ ]:
# twenty-third chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[22], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2009-01-01',
                                             'EDate':'2010-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data23_2009.csv', index=False)


In [ ]:
# twenty-fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[23], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2009-01-01',
                                             'EDate':'2010-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data24_2009.csv', index=False)


In [ ]:
# twenty-fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[24], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2009-01-01',
                                             'EDate':'2010-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data25_2009.csv', index=False)


In [ ]:
# twenty-sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[25], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2009-01-01',
                                             'EDate':'2010-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data26_2009.csv', index=False)


In [ ]:
# twenty-seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[26], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2009-01-01',
                                             'EDate':'2010-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data27_2009.csv', index=False)


In [ ]:
# twenty-eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[27], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2009-01-01',
                                             'EDate':'2010-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data28_2009.csv', index=False)


In [ ]:
# twenty-ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[28], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2009-01-01',
                                             'EDate':'2010-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data29_2009.csv', index=False)


In [ ]:
# thirtieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[29], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2009-01-01',
                                             'EDate':'2010-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data30_2009.csv', index=False)


### 2010

In [ ]:
# first chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2010-01-01',
                                             'EDate':'2011-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data01_2010.csv', index=False)


In [ ]:
# second chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2010-01-01',
                                             'EDate':'2011-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data02_2010.csv', index=False)


In [ ]:
# third chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2010-01-01',
                                             'EDate':'2011-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data03_2010.csv', index=False)


In [ ]:
# fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[3], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2010-01-01',
                                             'EDate':'2011-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data04_2010.csv', index=False)


In [ ]:
# fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[4], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2010-01-01',
                                             'EDate':'2011-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data05_2010.csv', index=False)


In [ ]:
# sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[5], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2010-01-01',
                                             'EDate':'2011-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data06_2010.csv', index=False)


In [ ]:
# seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[6], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2010-01-01',
                                             'EDate':'2011-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data07_2010.csv', index=False)


In [ ]:
# eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[7], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2010-01-01',
                                             'EDate':'2011-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data08_2010.csv', index=False)


In [ ]:
# ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[8], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2010-01-01',
                                             'EDate':'2011-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data09_2010.csv', index=False)


In [ ]:
# tenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[9], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2010-01-01',
                                             'EDate':'2011-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data10_2010.csv', index=False)


In [ ]:
# eleventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[10], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2010-01-01',
                                             'EDate':'2011-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data11_2010.csv', index=False)


In [ ]:
# twelfth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[11], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2010-01-01',
                                             'EDate':'2011-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data12_2010.csv', index=False)


In [ ]:
# thirteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[12], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2010-01-01',
                                             'EDate':'2011-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data13_2010.csv', index=False)


In [ ]:
# fourteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[13], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2010-01-01',
                                             'EDate':'2011-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data14_2010.csv', index=False)


In [ ]:
# fifteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[14], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2010-01-01',
                                             'EDate':'2011-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data15_2010.csv', index=False)


In [ ]:
# sixteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[15], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2010-01-01',
                                             'EDate':'2011-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data16_2010.csv', index=False)


In [ ]:
# seventeenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[16], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2010-01-01',
                                             'EDate':'2011-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data17_2010.csv', index=False)


In [ ]:
# eighteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[17], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2010-01-01',
                                             'EDate':'2011-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data18_2010.csv', index=False)


In [ ]:
# nineteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[18], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2010-01-01',
                                             'EDate':'2011-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data19_2010.csv', index=False)


In [ ]:
# twentieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[19], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2010-01-01',
                                             'EDate':'2011-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data20_2010.csv', index=False)


In [ ]:
# twenty-first chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[20], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2010-01-01',
                                             'EDate':'2011-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data21_2010.csv', index=False)


In [ ]:
# twenty-second chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[21], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2010-01-01',
                                             'EDate':'2011-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data22_2010.csv', index=False)


In [ ]:
# twenty-third chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[22], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2010-01-01',
                                             'EDate':'2011-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data23_2010.csv', index=False)


In [ ]:
# twenty-fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[23], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2010-01-01',
                                             'EDate':'2011-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data24_2010.csv', index=False)


In [ ]:
# twenty-fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[24], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2010-01-01',
                                             'EDate':'2011-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data25_2010.csv', index=False)


In [ ]:
# twenty-sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[25], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2010-01-01',
                                             'EDate':'2011-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data26_2010.csv', index=False)


In [ ]:
# twenty-seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[26], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2010-01-01',
                                             'EDate':'2011-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data27_2010.csv', index=False)


In [ ]:
# twenty-eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[27], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2010-01-01',
                                             'EDate':'2011-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data28_2010.csv', index=False)


In [ ]:
# twenty-ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[28], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2010-01-01',
                                             'EDate':'2011-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data29_2010.csv', index=False)


In [ ]:
# thirtieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[29], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2010-01-01',
                                             'EDate':'2011-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data30_2010.csv', index=False)


### 2011

In [ ]:
# first chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2011-01-01',
                                             'EDate':'2012-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data01_2011.csv', index=False)


In [ ]:
# second chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2011-01-01',
                                             'EDate':'2012-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data02_2011.csv', index=False)


In [ ]:
# third chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2011-01-01',
                                             'EDate':'2012-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data03_2011.csv', index=False)


In [ ]:
# fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[3], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2011-01-01',
                                             'EDate':'2012-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data04_2011.csv', index=False)


In [ ]:
# fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[4], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2011-01-01',
                                             'EDate':'2012-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data05_2011.csv', index=False)


In [ ]:
# sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[5], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2011-01-01',
                                             'EDate':'2012-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data06_2011.csv', index=False)


In [ ]:
# seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[6], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2011-01-01',
                                             'EDate':'2012-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data07_2011.csv', index=False)


In [ ]:
# eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[7], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2011-01-01',
                                             'EDate':'2012-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data08_2011.csv', index=False)


In [ ]:
# ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[8], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2011-01-01',
                                             'EDate':'2012-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data09_2011.csv', index=False)


In [ ]:
# tenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[9], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2011-01-01',
                                             'EDate':'2012-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data10_2011.csv', index=False)


In [ ]:
# eleventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[10], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2011-01-01',
                                             'EDate':'2012-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data11_2011.csv', index=False)


In [ ]:
# twelfth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[11], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2011-01-01',
                                             'EDate':'2012-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data12_2011.csv', index=False)


In [ ]:
# thirteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[12], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2011-01-01',
                                             'EDate':'2012-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data13_2011.csv', index=False)


In [ ]:
# fourteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[13], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2011-01-01',
                                             'EDate':'2012-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data14_2011.csv', index=False)


In [ ]:
# fifteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[14], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2011-01-01',
                                             'EDate':'2012-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data15_2011.csv', index=False)


In [ ]:
# sixteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[15], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2011-01-01',
                                             'EDate':'2012-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data16_2011.csv', index=False)


In [ ]:
# seventeenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[16], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2011-01-01',
                                             'EDate':'2012-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data17_2011.csv', index=False)


In [ ]:
# eighteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[17], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2011-01-01',
                                             'EDate':'2012-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data18_2011.csv', index=False)


In [ ]:
# nineteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[18], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2011-01-01',
                                             'EDate':'2012-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data19_2011.csv', index=False)


In [ ]:
# twentieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[19], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2011-01-01',
                                             'EDate':'2012-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data20_2011.csv', index=False)


In [ ]:
# twenty-first chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[20], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2011-01-01',
                                             'EDate':'2012-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data21_2011.csv', index=False)


In [ ]:
# twenty-second chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[21], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2011-01-01',
                                             'EDate':'2012-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data22_2011.csv', index=False)


In [ ]:
# twenty-third chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[22], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2011-01-01',
                                             'EDate':'2012-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data23_2011.csv', index=False)


In [ ]:
# twenty-fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[23], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2011-01-01',
                                             'EDate':'2012-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data24_2011.csv', index=False)


In [ ]:
# twenty-fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[24], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2011-01-01',
                                             'EDate':'2012-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data25_2011.csv', index=False)


In [ ]:
# twenty-sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[25], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2011-01-01',
                                             'EDate':'2012-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data26_2011.csv', index=False)


In [ ]:
# twenty-seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[26], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2011-01-01',
                                             'EDate':'2012-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data27_2011.csv', index=False)


In [ ]:
# twenty-eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[27], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2011-01-01',
                                             'EDate':'2012-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data28_2011.csv', index=False)


In [ ]:
# twenty-ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[28], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2011-01-01',
                                             'EDate':'2012-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data29_2011.csv', index=False)


In [ ]:
# thirtieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[29], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2011-01-01',
                                             'EDate':'2012-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data30_2011.csv', index=False)


### 2012

In [ ]:
# first chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2012-01-01',
                                             'EDate':'2013-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data01_2012.csv', index=False)


In [ ]:
# second chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2012-01-01',
                                             'EDate':'2013-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data02_2012.csv', index=False)


In [ ]:
# third chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2012-01-01',
                                             'EDate':'2013-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data03_2012.csv', index=False)


In [ ]:
# fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[3], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2012-01-01',
                                             'EDate':'2013-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data04_2012.csv', index=False)


In [ ]:
# fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[4], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2012-01-01',
                                             'EDate':'2013-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data05_2012.csv', index=False)


In [ ]:
# sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[5], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2012-01-01',
                                             'EDate':'2013-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data06_2012.csv', index=False)


In [ ]:
# seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[6], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2012-01-01',
                                             'EDate':'2013-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data07_2012.csv', index=False)


In [ ]:
# eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[7], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2012-01-01',
                                             'EDate':'2013-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data08_2012.csv', index=False)


In [ ]:
# ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[8], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2012-01-01',
                                             'EDate':'2013-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data09_2012.csv', index=False)


In [ ]:
# tenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[9], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2012-01-01',
                                             'EDate':'2013-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data10_2012.csv', index=False)


In [ ]:
# eleventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[10], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2012-01-01',
                                             'EDate':'2013-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data11_2012.csv', index=False)


In [ ]:
# twelfth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[11], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2012-01-01',
                                             'EDate':'2013-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data12_2012.csv', index=False)


In [ ]:
# thirteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[12], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2012-01-01',
                                             'EDate':'2013-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data13_2012.csv', index=False)


In [ ]:
# fourteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[13], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2012-01-01',
                                             'EDate':'2013-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data14_2012.csv', index=False)


In [ ]:
# fifteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[14], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2012-01-01',
                                             'EDate':'2013-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data15_2012.csv', index=False)


In [ ]:
# sixteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[15], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2012-01-01',
                                             'EDate':'2013-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data16_2012.csv', index=False)


In [ ]:
# seventeenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[16], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2012-01-01',
                                             'EDate':'2013-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data17_2012.csv', index=False)


In [ ]:
# eighteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[17], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2012-01-01',
                                             'EDate':'2013-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data18_2012.csv', index=False)


In [ ]:
# nineteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[18], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2012-01-01',
                                             'EDate':'2013-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data19_2012.csv', index=False)


In [ ]:
# twentieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[19], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2012-01-01',
                                             'EDate':'2013-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data20_2012.csv', index=False)


In [ ]:
# twenty-first chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[20], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2012-01-01',
                                             'EDate':'2013-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data21_2012.csv', index=False)


In [ ]:
# twenty-second chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[21], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2012-01-01',
                                             'EDate':'2013-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data22_2012.csv', index=False)


In [ ]:
# twenty-third chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[22], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2012-01-01',
                                             'EDate':'2013-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data23_2012.csv', index=False)


In [ ]:
# twenty-fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[23], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2012-01-01',
                                             'EDate':'2013-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data24_2012.csv', index=False)


In [ ]:
# twenty-fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[24], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2012-01-01',
                                             'EDate':'2013-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data25_2012.csv', index=False)


In [ ]:
# twenty-sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[25], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2012-01-01',
                                             'EDate':'2013-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data26_2012.csv', index=False)


In [ ]:
# twenty-seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[26], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2012-01-01',
                                             'EDate':'2013-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data27_2012.csv', index=False)


In [ ]:
# twenty-eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[27], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2012-01-01',
                                             'EDate':'2013-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data28_2012.csv', index=False)


In [ ]:
# twenty-ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[28], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2012-01-01',
                                             'EDate':'2013-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data29_2012.csv', index=False)


In [ ]:
# thirtieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[29], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2012-01-01',
                                             'EDate':'2013-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data30_2012.csv', index=False)


### 2013

In [ ]:
# first chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2013-01-01',
                                             'EDate':'2014-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data01_2013.csv', index=False)


In [ ]:
# second chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2013-01-01',
                                             'EDate':'2014-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data02_2013.csv', index=False)


In [ ]:
# third chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2013-01-01',
                                             'EDate':'2014-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data03_2013.csv', index=False)


In [ ]:
# fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[3], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2013-01-01',
                                             'EDate':'2014-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data04_2013.csv', index=False)


In [ ]:
# fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[4], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2013-01-01',
                                             'EDate':'2014-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data05_2013.csv', index=False)


In [ ]:
# sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[5], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2013-01-01',
                                             'EDate':'2014-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data06_2013.csv', index=False)


In [ ]:
# seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[6], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2013-01-01',
                                             'EDate':'2014-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data07_2013.csv', index=False)


In [ ]:
# eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[7], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2013-01-01',
                                             'EDate':'2014-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data08_2013.csv', index=False)


In [ ]:
# ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[8], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2013-01-01',
                                             'EDate':'2014-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data09_2013.csv', index=False)


In [ ]:
# tenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[9], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2013-01-01',
                                             'EDate':'2014-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data10_2013.csv', index=False)


In [ ]:
# eleventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[10], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2013-01-01',
                                             'EDate':'2014-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data11_2013.csv', index=False)


In [ ]:
# twelfth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[11], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2013-01-01',
                                             'EDate':'2014-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data12_2013.csv', index=False)


In [ ]:
# thirteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[12], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2013-01-01',
                                             'EDate':'2014-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data13_2013.csv', index=False)


In [ ]:
# fourteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[13], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2013-01-01',
                                             'EDate':'2014-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data14_2013.csv', index=False)


In [ ]:
# fifteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[14], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2013-01-01',
                                             'EDate':'2014-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data15_2013.csv', index=False)


In [ ]:
# sixteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[15], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2013-01-01',
                                             'EDate':'2014-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data16_2013.csv', index=False)


In [ ]:
# seventeenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[16], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2013-01-01',
                                             'EDate':'2014-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data17_2013.csv', index=False)


In [ ]:
# eighteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[17], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2013-01-01',
                                             'EDate':'2014-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data18_2013.csv', index=False)


In [ ]:
# nineteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[18], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2013-01-01',
                                             'EDate':'2014-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data19_2013.csv', index=False)


In [ ]:
# twentieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[19], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2013-01-01',
                                             'EDate':'2014-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data20_2013.csv', index=False)


In [ ]:
# twenty-first chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[20], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2013-01-01',
                                             'EDate':'2014-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data21_2013.csv', index=False)


In [ ]:
# twenty-second chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[21], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2013-01-01',
                                             'EDate':'2014-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data22_2013.csv', index=False)


In [ ]:
# twenty-third chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[22], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2013-01-01',
                                             'EDate':'2014-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data23_2013.csv', index=False)


In [ ]:
# twenty-fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[23], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2013-01-01',
                                             'EDate':'2014-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data24_2013.csv', index=False)


In [ ]:
# twenty-fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[24], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2013-01-01',
                                             'EDate':'2014-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data25_2013.csv', index=False)


In [ ]:
# twenty-sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[25], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2013-01-01',
                                             'EDate':'2014-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data26_2013.csv', index=False)


In [ ]:
# twenty-seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[26], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2013-01-01',
                                             'EDate':'2014-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data27_2013.csv', index=False)


In [ ]:
# twenty-eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[27], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2013-01-01',
                                             'EDate':'2014-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data28_2013.csv', index=False)


In [ ]:
# twenty-ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[28], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2013-01-01',
                                             'EDate':'2014-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data29_2013.csv', index=False)


In [ ]:
# thirtieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[29], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2013-01-01',
                                             'EDate':'2014-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data30_2013.csv', index=False)


### 2014

In [ ]:
# first chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2014-01-01',
                                             'EDate':'2015-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data01_2014.csv', index=False)


In [ ]:
# second chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2014-01-01',
                                             'EDate':'2015-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data02_2014.csv', index=False)


In [ ]:
# third chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2014-01-01',
                                             'EDate':'2015-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data03_2014.csv', index=False)


In [ ]:
# fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[3], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2014-01-01',
                                             'EDate':'2015-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data04_2014.csv', index=False)


In [ ]:
# fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[4], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2014-01-01',
                                             'EDate':'2015-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data05_2014.csv', index=False)


In [ ]:
# sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[5], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2014-01-01',
                                             'EDate':'2015-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data06_2014.csv', index=False)


In [ ]:
# seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[6], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2014-01-01',
                                             'EDate':'2015-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data07_2014.csv', index=False)


In [ ]:
# eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[7], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2014-01-01',
                                             'EDate':'2015-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data08_2014.csv', index=False)


In [ ]:
# ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[8], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2014-01-01',
                                             'EDate':'2015-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data09_2014.csv', index=False)


In [ ]:
# tenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[9], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2014-01-01',
                                             'EDate':'2015-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data10_2014.csv', index=False)


In [ ]:
# eleventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[10], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2014-01-01',
                                             'EDate':'2015-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data11_2014.csv', index=False)


In [ ]:
# twelfth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[11], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2014-01-01',
                                             'EDate':'2015-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data12_2014.csv', index=False)


In [ ]:
# thirteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[12], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2014-01-01',
                                             'EDate':'2015-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data13_2014.csv', index=False)


In [ ]:
# fourteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[13], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2014-01-01',
                                             'EDate':'2015-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data14_2014.csv', index=False)


In [ ]:
# fifteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[14], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2014-01-01',
                                             'EDate':'2015-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data15_2014.csv', index=False)


In [ ]:
# sixteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[15], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2014-01-01',
                                             'EDate':'2015-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data16_2014.csv', index=False)


In [ ]:
# seventeenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[16], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2014-01-01',
                                             'EDate':'2015-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data17_2014.csv', index=False)


In [ ]:
# eighteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[17], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2014-01-01',
                                             'EDate':'2015-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data18_2014.csv', index=False)


In [ ]:
# nineteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[18], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2014-01-01',
                                             'EDate':'2015-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data19_2014.csv', index=False)


In [ ]:
# twentieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[19], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2014-01-01',
                                             'EDate':'2015-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data20_2014.csv', index=False)


In [ ]:
# twenty-first chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[20], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2014-01-01',
                                             'EDate':'2015-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data21_2014.csv', index=False)


In [ ]:
# twenty-second chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[21], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2014-01-01',
                                             'EDate':'2015-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data22_2014.csv', index=False)


In [ ]:
# twenty-third chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[22], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2014-01-01',
                                             'EDate':'2015-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data23_2014.csv', index=False)


In [ ]:
# twenty-fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[23], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2014-01-01',
                                             'EDate':'2015-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data24_2014.csv', index=False)


In [ ]:
# twenty-fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[24], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2014-01-01',
                                             'EDate':'2015-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data25_2014.csv', index=False)


In [ ]:
# twenty-sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[25], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2014-01-01',
                                             'EDate':'2015-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data26_2014.csv', index=False)


In [ ]:
# twenty-seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[26], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2014-01-01',
                                             'EDate':'2015-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data27_2014.csv', index=False)


In [ ]:
# twenty-eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[27], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2014-01-01',
                                             'EDate':'2015-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data28_2014.csv', index=False)


In [ ]:
# twenty-ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[28], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2014-01-01',
                                             'EDate':'2015-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data29_2014.csv', index=False)


In [ ]:
# thirtieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[29], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2014-01-01',
                                             'EDate':'2015-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data30_2014.csv', index=False)


### 2015

In [ ]:
# first chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2015-01-01',
                                             'EDate':'2016-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data01_2015.csv', index=False)


In [ ]:
# second chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2015-01-01',
                                             'EDate':'2016-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data02_2015.csv', index=False)


In [ ]:
# third chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2015-01-01',
                                             'EDate':'2016-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data03_2015.csv', index=False)


In [ ]:
# fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[3], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2015-01-01',
                                             'EDate':'2016-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data04_2015.csv', index=False)


In [ ]:
# fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[4], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2015-01-01',
                                             'EDate':'2016-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data05_2015.csv', index=False)


In [ ]:
# sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[5], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2015-01-01',
                                             'EDate':'2016-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data06_2015.csv', index=False)


In [ ]:
# seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[6], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2015-01-01',
                                             'EDate':'2016-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data07_2015.csv', index=False)


In [ ]:
# eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[7], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2015-01-01',
                                             'EDate':'2016-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data08_2015.csv', index=False)


In [ ]:
# ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[8], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2015-01-01',
                                             'EDate':'2016-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data09_2015.csv', index=False)


In [ ]:
# tenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[9], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2015-01-01',
                                             'EDate':'2016-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data10_2015.csv', index=False)


In [ ]:
# eleventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[10], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2015-01-01',
                                             'EDate':'2016-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data11_2015.csv', index=False)


In [ ]:
# twelfth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[11], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2015-01-01',
                                             'EDate':'2016-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data12_2015.csv', index=False)


In [ ]:
# thirteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[12], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2015-01-01',
                                             'EDate':'2016-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data13_2015.csv', index=False)


In [ ]:
# fourteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[13], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2015-01-01',
                                             'EDate':'2016-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data14_2015.csv', index=False)


In [ ]:
# fifteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[14], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2015-01-01',
                                             'EDate':'2016-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data15_2015.csv', index=False)


In [ ]:
# sixteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[15], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2015-01-01',
                                             'EDate':'2016-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data16_2015.csv', index=False)


In [ ]:
# seventeenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[16], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2015-01-01',
                                             'EDate':'2016-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data17_2015.csv', index=False)


In [ ]:
# eighteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[17], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2015-01-01',
                                             'EDate':'2016-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data18_2015.csv', index=False)


In [ ]:
# nineteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[18], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2015-01-01',
                                             'EDate':'2016-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data19_2015.csv', index=False)


In [ ]:
# twentieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[19], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2015-01-01',
                                             'EDate':'2016-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data20_2015.csv', index=False)


In [ ]:
# twenty-first chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[20], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2015-01-01',
                                             'EDate':'2016-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data21_2015.csv', index=False)


In [ ]:
# twenty-second chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[21], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2015-01-01',
                                             'EDate':'2016-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data22_2015.csv', index=False)


In [ ]:
# twenty-third chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[22], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2015-01-01',
                                             'EDate':'2016-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data23_2015.csv', index=False)


In [ ]:
# twenty-fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[23], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2015-01-01',
                                             'EDate':'2016-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data24_2015.csv', index=False)


In [ ]:
# twenty-fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[24], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2015-01-01',
                                             'EDate':'2016-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data25_2015.csv', index=False)


In [ ]:
# twenty-sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[25], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2015-01-01',
                                             'EDate':'2016-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data26_2015.csv', index=False)


In [ ]:
# twenty-seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[26], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2015-01-01',
                                             'EDate':'2016-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data27_2015.csv', index=False)


In [ ]:
# twenty-eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[27], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2015-01-01',
                                             'EDate':'2016-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data28_2015.csv', index=False)


In [ ]:
# twenty-ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[28], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2015-01-01',
                                             'EDate':'2016-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data29_2015.csv', index=False)


In [ ]:
# thirtieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[29], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2015-01-01',
                                             'EDate':'2016-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data30_2015.csv', index=False)


### 2016

In [ ]:
# first chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2016-01-01',
                                             'EDate':'2017-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data01_2016.csv', index=False)


In [ ]:
# second chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2016-01-01',
                                             'EDate':'2017-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data02_2016.csv', index=False)


In [ ]:
# third chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2016-01-01',
                                             'EDate':'2017-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data03_2016.csv', index=False)


In [ ]:
# fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[3], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2016-01-01',
                                             'EDate':'2017-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data04_2016.csv', index=False)


In [ ]:
# fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[4], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2016-01-01',
                                             'EDate':'2017-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data05_2016.csv', index=False)


In [ ]:
# sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[5], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2016-01-01',
                                             'EDate':'2017-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data06_2016.csv', index=False)


In [ ]:
# seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[6], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2016-01-01',
                                             'EDate':'2017-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data07_2016.csv', index=False)


In [ ]:
# eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[7], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2016-01-01',
                                             'EDate':'2017-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data08_2016.csv', index=False)


In [ ]:
# ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[8], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2016-01-01',
                                             'EDate':'2017-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data09_2016.csv', index=False)


In [ ]:
# tenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[9], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2016-01-01',
                                             'EDate':'2017-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data10_2016.csv', index=False)


In [ ]:
# eleventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[10], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2016-01-01',
                                             'EDate':'2017-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data11_2016.csv', index=False)


In [ ]:
# twelfth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[11], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2016-01-01',
                                             'EDate':'2017-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data12_2016.csv', index=False)


In [ ]:
# thirteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[12], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2016-01-01',
                                             'EDate':'2017-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data13_2016.csv', index=False)


In [ ]:
# fourteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[13], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2016-01-01',
                                             'EDate':'2017-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data14_2016.csv', index=False)


In [ ]:
# fifteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[14], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2016-01-01',
                                             'EDate':'2017-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data15_2016.csv', index=False)


In [ ]:
# sixteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[15], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2016-01-01',
                                             'EDate':'2017-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data16_2016.csv', index=False)


In [ ]:
# seventeenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[16], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2016-01-01',
                                             'EDate':'2017-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data17_2016.csv', index=False)


In [ ]:
# eighteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[17], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2016-01-01',
                                             'EDate':'2017-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data18_2016.csv', index=False)


In [ ]:
# nineteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[18], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2016-01-01',
                                             'EDate':'2017-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data19_2016.csv', index=False)


In [ ]:
# twentieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[19], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2016-01-01',
                                             'EDate':'2017-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data20_2016.csv', index=False)


In [ ]:
# twenty-first chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[20], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2016-01-01',
                                             'EDate':'2017-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data21_2016.csv', index=False)


In [ ]:
# twenty-second chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[21], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2016-01-01',
                                             'EDate':'2017-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data22_2016.csv', index=False)


In [ ]:
# twenty-third chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[22], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2016-01-01',
                                             'EDate':'2017-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data23_2016.csv', index=False)


In [ ]:
# twenty-fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[23], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2016-01-01',
                                             'EDate':'2017-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data24_2016.csv', index=False)


In [ ]:
# twenty-fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[24], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2016-01-01',
                                             'EDate':'2017-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data25_2016.csv', index=False)


In [ ]:
# twenty-sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[25], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2016-01-01',
                                             'EDate':'2017-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data26_2016.csv', index=False)


In [ ]:
# twenty-seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[26], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2016-01-01',
                                             'EDate':'2017-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data27_2016.csv', index=False)


In [ ]:
# twenty-eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[27], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2016-01-01',
                                             'EDate':'2017-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data28_2016.csv', index=False)


In [ ]:
# twenty-ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[28], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2016-01-01',
                                             'EDate':'2017-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data29_2016.csv', index=False)


In [ ]:
# thirtieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[29], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2016-01-01',
                                             'EDate':'2017-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data30_2016.csv', index=False)


### 2017

In [ ]:
# first chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2017-01-01',
                                             'EDate':'2018-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data01_2017.csv', index=False)


In [ ]:
# second chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2017-01-01',
                                             'EDate':'2018-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data02_2017.csv', index=False)


In [ ]:
# third chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2017-01-01',
                                             'EDate':'2018-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data03_2017.csv', index=False)


In [ ]:
# fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[3], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2017-01-01',
                                             'EDate':'2018-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data04_2017.csv', index=False)


In [ ]:
# fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[4], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2017-01-01',
                                             'EDate':'2018-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data05_2017.csv', index=False)


In [ ]:
# sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[5], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2017-01-01',
                                             'EDate':'2018-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data06_2017.csv', index=False)


In [ ]:
# seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[6], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2017-01-01',
                                             'EDate':'2018-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data07_2017.csv', index=False)


In [ ]:
# eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[7], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2017-01-01',
                                             'EDate':'2018-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data08_2017.csv', index=False)


In [ ]:
# ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[8], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2017-01-01',
                                             'EDate':'2018-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data09_2017.csv', index=False)


In [ ]:
# tenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[9], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2017-01-01',
                                             'EDate':'2018-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data10_2017.csv', index=False)


In [ ]:
# eleventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[10], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2017-01-01',
                                             'EDate':'2018-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data11_2017.csv', index=False)


In [ ]:
# twelfth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[11], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2017-01-01',
                                             'EDate':'2018-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data12_2017.csv', index=False)


In [ ]:
# thirteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[12], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2017-01-01',
                                             'EDate':'2018-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data13_2017.csv', index=False)


In [ ]:
# fourteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[13], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2017-01-01',
                                             'EDate':'2018-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data14_2017.csv', index=False)


In [ ]:
# fifteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[14], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2017-01-01',
                                             'EDate':'2018-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data15_2017.csv', index=False)


In [ ]:
# sixteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[15], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2017-01-01',
                                             'EDate':'2018-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data16_2017.csv', index=False)


In [ ]:
# seventeenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[16], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2017-01-01',
                                             'EDate':'2018-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data17_2017.csv', index=False)


In [ ]:
# eighteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[17], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2017-01-01',
                                             'EDate':'2018-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data18_2017.csv', index=False)


In [ ]:
# nineteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[18], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2017-01-01',
                                             'EDate':'2018-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data19_2017.csv', index=False)


In [ ]:
# twentieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[19], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2017-01-01',
                                             'EDate':'2018-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data20_2017.csv', index=False)


In [ ]:
# twenty-first chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[20], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2017-01-01',
                                             'EDate':'2018-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data21_2017.csv', index=False)


In [ ]:
# twenty-second chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[21], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2017-01-01',
                                             'EDate':'2018-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data22_2017.csv', index=False)


In [ ]:
# twenty-third chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[22], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2017-01-01',
                                             'EDate':'2018-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data23_2017.csv', index=False)


In [ ]:
# twenty-fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[23], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2017-01-01',
                                             'EDate':'2018-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data24_2017.csv', index=False)


In [ ]:
# twenty-fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[24], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2017-01-01',
                                             'EDate':'2018-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data25_2017.csv', index=False)


In [ ]:
# twenty-sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[25], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2017-01-01',
                                             'EDate':'2018-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data26_2017.csv', index=False)


In [ ]:
# twenty-seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[26], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2017-01-01',
                                             'EDate':'2018-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data27_2017.csv', index=False)


In [ ]:
# twenty-eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[27], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2017-01-01',
                                             'EDate':'2018-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data28_2017.csv', index=False)


In [ ]:
# twenty-ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[28], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2017-01-01',
                                             'EDate':'2018-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data29_2017.csv', index=False)


In [ ]:
# thirtieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[29], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2017-01-01',
                                             'EDate':'2018-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data30_2017.csv', index=False)


### 2018

In [ ]:
# first chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[0], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2018-01-01',
                                             'EDate':'2019-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data01_2018.csv', index=False)


In [ ]:
# second chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[1], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2018-01-01',
                                             'EDate':'2019-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data02_2018.csv', index=False)


In [ ]:
# third chuck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[2], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2018-01-01',
                                             'EDate':'2019-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data03_2018.csv', index=False)


In [ ]:
# fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[3], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2018-01-01',
                                             'EDate':'2019-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data04_2018.csv', index=False)


In [ ]:
# fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[4], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2018-01-01',
                                             'EDate':'2019-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data05_2018.csv', index=False)


In [ ]:
# sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[5], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2018-01-01',
                                             'EDate':'2019-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data06_2018.csv', index=False)


In [ ]:
# seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[6], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2018-01-01',
                                             'EDate':'2019-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data07_2018.csv', index=False)


In [ ]:
# eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[7], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2018-01-01',
                                             'EDate':'2019-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data08_2018.csv', index=False)


In [ ]:
# ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[8], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2018-01-01',
                                             'EDate':'2019-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data09_2018.csv', index=False)


In [ ]:
# tenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[9], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2018-01-01',
                                             'EDate':'2019-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data10_2018.csv', index=False)


In [ ]:
# eleventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[10], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2018-01-01',
                                             'EDate':'2019-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data11_2018.csv', index=False)


In [ ]:
# twelfth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[11], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2018-01-01',
                                             'EDate':'2019-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data12_2018.csv', index=False)


In [ ]:
# thirteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[12], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2018-01-01',
                                             'EDate':'2019-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data13_2018.csv', index=False)


In [ ]:
# fourteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[13], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2018-01-01',
                                             'EDate':'2019-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data14_2018.csv', index=False)


In [ ]:
# fifteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[14], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2018-01-01',
                                             'EDate':'2019-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data15_2018.csv', index=False)


In [ ]:
# sixteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[15], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2018-01-01',
                                             'EDate':'2019-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data16_2018.csv', index=False)


In [ ]:
# seventeenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[16], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2018-01-01',
                                             'EDate':'2019-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data17_2018.csv', index=False)


In [ ]:
# eighteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[17], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2018-01-01',
                                             'EDate':'2019-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data18_2018.csv', index=False)


In [ ]:
# nineteenth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[18], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2018-01-01',
                                             'EDate':'2019-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data19_2018.csv', index=False)


In [ ]:
# twentieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[19], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2018-01-01',
                                             'EDate':'2019-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data20_2019.csv', index=False)


In [ ]:
# twenty-first chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[20], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2018-01-01',
                                             'EDate':'2019-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data21_2018.csv', index=False)


In [ ]:
# twenty-second chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[21], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2018-01-01',
                                             'EDate':'2019-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data22_2018.csv', index=False)


In [ ]:
# twenty-third chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[22], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2018-01-01',
                                             'EDate':'2019-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data23_2018.csv', index=False)


In [ ]:
# twenty-fourth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[23], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2018-01-01',
                                             'EDate':'2019-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data24_2018.csv', index=False)


In [ ]:
# twenty-fifth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[24], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2018-01-01',
                                             'EDate':'2019-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data25_2018.csv', index=False)


In [ ]:
# twenty-sixth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[25], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2018-01-01',
                                             'EDate':'2019-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data26_2018.csv', index=False)


In [ ]:
# twenty-seventh chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[26], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2018-01-01',
                                             'EDate':'2019-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data27_2018.csv', index=False)


In [ ]:
# twenty-eighth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[27], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2018-01-01',
                                             'EDate':'2019-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data28_2018.csv', index=False)


In [ ]:
# twenty-ninth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[28], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2018-01-01',
                                             'EDate':'2019-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data29_2018.csv', index=False)


In [ ]:
# thirtieth chunck of the healthy data
healthy_data, err = ek.get_data(healthy_CIK_list[29], fields=fields,
                                 parameters={'Frq':'Y',
                                             'SDate':'2018-01-01',
                                             'EDate':'2019-01-01'})
# and save the dataframe as a csv file
healthy_data.to_csv('data_healthy_CIK/healthy_data30_2018.csv', index=False)
